### Integration with ACT

Step 1: Load data from timeloop_summary.yaml, generated by m00_time-loop_extract.ipynb.

In [1]:
import yaml

def load_timeloop_summary_as_dict(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    return data

summary = load_timeloop_summary_as_dict('timeloop_summary.yaml')
# DRAM capacities
total_cap = summary['dram_utilized_capacity']

# Total chip area and energy
total_area = summary['total_chip_area_um2']
total_energy = summary['total_energy_uJ']

# Total cycles
total_cycles = summary['total_cycles']

# Print them
print("Total DRAM Capacity (Byte) :", total_cap)
print("Total Chip Area (um^2)     :", total_area)
print("Total Energy (uJ)          :", total_energy)
print("Total Cycles               :", total_cycles)


Total DRAM Capacity (Byte) : 368547072.0
Total Chip Area (um^2)     : 238279.12
Total Energy (uJ)          : 291592.9399999992
Total Cycles               : 2754609152


In [2]:
import json
import sys
import os

def add_act_tutorial_to_path(path="/home/workspace/ACT-tutorial"):
    """Adds the ACT-tutorial directory to the Python module path."""
    if path not in sys.path:
        sys.path.append(path)

    # Optional: confirm path exists
    if not os.path.exists(path):
        raise FileNotFoundError(f"Provided path does not exist: {path}")

add_act_tutorial_to_path("/home/workspace/ACT-tutorial")

from dram_model import Fab_DRAM
from hdd_model import Fab_HDD
from ssd_model import Fab_SSD
from logic_model  import Fab_Logic


In [3]:
# Assume parameters
# Refernce: https://www.extremetech.com/computing/data-center-ai-gpus-may-have-extremely-short-lifespans
use_ci     = "loc_usa"
Kr         = 0.15   # kg CO2
T          = 1.8    # years
LT         = 3      # years
clock_freq = 200    # MHz

# Initialize components
processor = Fab_Logic(process_node=28, gpa="95", fab_carbon_intensity="loc_taiwan", use_carbon_intensity="loc_usa",debug=False, fab_yield=0.8)
DRAM      = Fab_DRAM(config="lpddr4",fab_yield=0.8)
N         = 2
n_run     = (T *365*24*60*60) / ((1/(clock_freq*1000000))*total_cycles)

# Input Timeloop output to to ACT
total_area_cm2 = total_area / 100000000
total_cap_GB = total_cap/1000000000
total_energy_kWh = total_energy / (3600*10**9)
processor.set_area(total_area_cm2)
DRAM.set_capacity(total_cap_GB)

# Calculate Carbon Footprint (kg Co2)
cf_package      = N * Kr
cf_processor    = processor.get_carbon() / 1000.
cf_DRAM         = DRAM.get_carbon() / 1000.
cf_embodied     = cf_package + cf_processor + cf_DRAM
cf_dis_embodied = T/LT*cf_embodied
cf_operational  = processor.get_use_ci() * n_run * total_energy_kWh / 1000.

# Final Carbon Footprint
cf_final = cf_operational + cf_dis_embodied

In [4]:
print("-----------------------------------------------")
print("Total Carbon Footprint    : %5f"%(cf_final), "kg CO2")
print("- Operational CF          : %5f"%(cf_operational), "kg CO2")
print("- Discounted Embodied CF  : %5f"%(cf_dis_embodied), "kg CO2")
print()
print("- Embodied CF             : %5f"%(cf_embodied), "kg CO2")
print("   - Processor CF         : %5f"%(cf_processor), "kg CO2")
print("   - DRAM CF              : %5f"%(cf_DRAM), "kg CO2")
print("   - Package CF           : %5f"%(cf_package), "kg CO2")
print("-----------------------------------------------")

-----------------------------------------------
Total Carbon Footprint    : 0.322267 kg CO2
- Operational CF          : 0.126855 kg CO2
- Discounted Embodied CF  : 0.195412 kg CO2

- Embodied CF             : 0.325686 kg CO2
   - Processor CF         : 0.003573 kg CO2
   - DRAM CF              : 0.022113 kg CO2
   - Package CF           : 0.300000 kg CO2
-----------------------------------------------
